<a href="https://githubtocolab.com/Eunseob/purdue_me597/blob/main/lab/lab10/PL10_Colab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prelab 10.3 Machine Learning 3 - Real-time Sound Recognition for Classification

# Load CNN model on Raspberry Pi

## 3.1 librosa installation on Raspberry Pi

In Lab 10, we will implement sound recognition on Raspberry Pi using the CNN model in the previous section. The librosa package will be used for the feature extraction of the CNN model. Follow the steps below to install librosa on your Raspberry Pi. If you have not installed TensorFlow version 2.2.0 on Raspberry Pi yet, please go to [Prelab 9.1](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/lab/lab9/PL9_Colab1.ipynb) and install it. This will take approximately 10 minutes. Note that please run the commands step-by-step not to being crashed on Raspberry Pi because of the package compatibility.

<br>

1. To make Raspberry Pi packages up to date, run the command lines below.

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_RaspberryPi.png?raw=tru" width="20">Raspberry Pi - Terminal**

`sudo apt update`

`sudo apt upgrade -y`

---

<br>

2. Install and setup dependent Raspberry Pi packages

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_RaspberryPi.png?raw=tru" width="20">Raspberry Pi - Terminal**

`sudo apt install llvm-**X**`

`LLVM_CONFIG=llvm-config-9 pip3 install llvmlite==0.36`

---

<br>

3. Install compatible Python packages for librosa

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_RaspberryPi.png?raw=tru" width="20">Raspberry Pi - Terminal**

`pip3 install numpy==1.21.0`

`pip3 install numba==0.53`

`pip3 install resampy==0.4.2 --no-deps`

`pip3 install colorama==0.3.9`

---

<br>

4. Finally, install librosa

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_RaspberryPi.png?raw=tru" width="20">Raspberry Pi - Terminal**

`pip3 install librosa==0.9.2`

---

<br></br>

Now your are ready to use librosa on Raspberry pi. Perform Task 1 to see if librosa is installed successfully on your Raspberry Pi.


### Task 3.1

By running the example Python script below, load librosa and check the version as Figure 12 below and then attach it to the report.



---

Place your screenshot for Task 3.1 here.

---



**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_RaspberryPi.png?raw=tru" width="20">Raspberry Pi - Terminal & Python**

```
$ python3
>>> import librosa
>>> print(“librosa version is {}.”.format(librosa.__version__)) 
```

---

<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/prelab10_fig12.png?raw=true" width="50%">

*Figure 12 librosa version check*

## 3.2 Load CNN model on Raspberry Pi

Let's load and test the CNN model which you finally developed on Raspberry Pi. Please follow the steps below.

<br>

  1. Create a working directory which your model and Python file will be in.
     
     a. The directory example (full path): ‘/home/pi/prelab10’

  2. Copy and paste your final model binary file (*.h5 format) in the previous section to the directory (‘/home/pi/prelab10’ in this example).
     
     a. In this example, the model filename is '20230306_043352_Prelab10_CNN_model1.h5'.
  
  3. Create a Python file as 'prelab10_load_model.py' on the same directory.


<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/prelab10_img3.png?raw=true" width="80%">


  4. Code the Python script below in the created Python file ('prelab10_load_model.py') and then save it.

---

**<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/icon_Python.png?raw=tru" width="20">Python - Python3 (prelab10_load_model.py)**

```
import tensorflow as tf

model = tf.keras.models.load_model('20230306_043352_Prelab10_CNN_model1.h5',  compile=False) # load your model

print(model.summary()) # print out model (CNN) summary

```

---

Now, you are ready to use the model you created in Lab 10. Perform Task 3.2 below to see if the model is loaded successfully on your Raspberry Pi.


### Task 3.2

  1. After performing the stpes above, run the Python file. Is the result as you expected?

  2. Capture the result of Terminal or Thonny Shell as Figure 13 and attach it to the report.

    ---

    Place your screenshot here.

    ---


<img src="https://github.com/Eunseob/purdue_me597/blob/main/lab/img/prelab10_fig13.png?raw=true" width="60%">

*Figure 2 Check the model summary*

<br></br>

Please continue to [Prelab 10.3 here](https://colab.research.google.com/github/Eunseob/purdue_me597/blob/main/lab/lab10/PL10_Colab3.ipynb).
